# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**В рамках исследования ответим на несколько вопросов:**

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?
    
В заключение сформулируем общий вывод по результатам исследования.

## Шаг 1. Загрузка и первичный обзор данных

In [1]:
# сначала подключим библиотеки, которые понадобятся нам в процессе анализа данных
import pandas as pd # импортируем pandas - основной пакет, используемый при анализе
from pymystem3 import Mystem # импортируем лемматизатор для слов на русском
from collections import Counter # модуль Counter для подсчёта количества лем
m = Mystem()

In [2]:
# загрузим файл с данными 
df = pd.read_csv(input())
df.info() # и изучим общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:

*    `children` — количество детей в семье
*    `days_employed` — общий трудовой стаж в днях
*    `dob_years` — возраст клиента в годах
*    `education` — уровень образования клиента
*    `education_id` — идентификатор уровня образования
*    `family_status` — семейное положение
*    `family_status_id` — идентификатор семейного положения
*    `gender` — пол клиента
*    `income_type` — тип занятости
*    `debt` — имел ли задолженность по возврату кредитов
*    `total_income` — ежемесячный доход
*    `purpose` — цель получения кредита

In [3]:
# Смотрю на первые 10 строк датафрейма - хотелось бы сразу увидеть, что необычного в столбцах days_employed и total_income
display(df.head(10))

# заодно взглянем и на последние 5
display(df.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

В предоставленном для анализа файле:

* 12 столбцов (переменных)
* 21525 строк (записей/наблюдений)
    
Целочисленных и строковых переменных в таблице поровну: по 5 каждого вида. Ещё два столбца - **трудовой стаж в днях** и **общий доход** - вещественные числа.
Стиль в названии колонок не нарушен.

Вещественный тип данных в столбце `days_employed` вызывает подозрения - хранение дробных дней выглядит излишним.
При этом, в обоих столбцах не достаёт около двух тысяч наблюдений - вероятно, пропущенные данные хранятся как "вещественные числа" NaN.

Первые и последние строки таблицы не дали прямого ответа о наличии NaN в двух интересующих нас переменных, но ярко обозначили **наличие отрицательных значений** в общем трудовом стаже в днях. Также следует обратить внимание на слишком большие (по модулю) значения в этой переменной: уже в 4 строке наблюдаем запись с 340266 днями стажа - это 932 года. Значения нелогичны - нужно будет перепроверить. Гипотеза: стаж представлен в часах, а не днях.

Обращает на себя внимание и столбец "**education**" - значения в нём не приведены к одному регистру.

В каждой строке таблицы — данные об отдельном клиента банка. Большая часть колонок описывает самого клиента: возраст, пол, образование и т.д. Столбец `purpose` содержит информацию о цели кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но в данных видны проблемы: отрицательные значения стажа, пропуски, разный регистр значений в строковых переменных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала подсчитаем количество пропусков в каждой колонке.

In [4]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Равное количество пропусков в переменных `days_employed` и `total_income` наводит на мысль о том, что пропуски неслучайны. Взглянем на строки, в которых есть пропуски.

In [5]:
display(df[df['days_employed'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Дополнительно проверим, во всех ли строках с пропуском стажа пропущена информация о доходе.

In [6]:
print(df[df['days_employed'].isna()]['total_income'].unique())

[nan]


Пропуски неслучайны - NaN в стаже соответствует NaN в ежемесячном доходе, во всех случаях без исключения. 

Заполним пропуски - для каждой строки знаем тип дохода и можем использовать эту информацию для поиска подходящего значения.
Заполнить значение дохода можем сразу, тогда как для заполнения трудового стажа необходимо будет в первую очередь выполнить предобработку этого столбца.

In [7]:
median_income_by_income_type = df.sort_values('total_income').groupby('income_type')['total_income'].median().rename('median')
mean_income_by_income_type = df.groupby('income_type')['total_income'].mean().rename('mean')
income_by_type_diff = (mean_income_by_income_type - median_income_by_income_type).rename('mean - median')

print(pd.concat([median_income_by_income_type, mean_income_by_income_type, income_by_type_diff], axis=1))

                        median           mean  mean - median
income_type                                                 
безработный      131339.751676  131339.751676       0.000000
в декрете         53829.130729   53829.130729       0.000000
госслужащий      150447.935283  170898.309923   20450.374640
компаньон        172357.950966  202417.461462   30059.510496
пенсионер        118514.486412  137127.465690   18612.979279
предприниматель  499163.144947  499163.144947       0.000000
сотрудник        142594.396847  161380.260488   18785.863640
студент           98201.625314   98201.625314       0.000000


Видим, что среднее значение дохода выше среди госслужащих, компаньонов, пенсионеров и сотрудников. Показатель среднего чувствительнее к выбросам - более логичным выбором для заполнения кажется медианное значение, воспользуемся им.

In [8]:
for value in df['income_type'].unique():
    df.loc[df.loc[:,'income_type'] == value, 'total_income'] = df.loc[df.loc[:,'income_type'] == value, 'total_income'].fillna(median_income_by_income_type[value])

Проверим результат заполнения - убедимся, что пропущенных значений в столбце `total_income` не осталось, а также в том, что медианное значение не изменилось.

In [9]:
median_income_by_filled_income_type = df.sort_values('total_income').groupby('income_type')['total_income'].median().rename('new_median')

print(f"В столбце `total_income` {df['total_income'].isna().sum()} нулевых значений")
print()
print(pd.concat([median_income_by_income_type.rename('original_median'), median_income_by_filled_income_type, (median_income_by_income_type - median_income_by_filled_income_type).rename('difference')], axis=1))

В столбце `total_income` 0 нулевых значений

                 original_median     new_median  difference
income_type                                                
безработный        131339.751676  131339.751676         0.0
в декрете           53829.130729   53829.130729         0.0
госслужащий        150447.935283  150447.935283         0.0
компаньон          172357.950966  172357.950966         0.0
пенсионер          118514.486412  118514.486412         0.0
предприниматель    499163.144947  499163.144947         0.0
сотрудник          142594.396847  142594.396847         0.0
студент             98201.625314   98201.625314         0.0


Заполнение пропусков в столбце `income_type` выполнено успешно. После обработки значений стажа работы воспользуемся тем же методом для заполнения пропусков в нём.

**Вывод**

В предоставленных данных обнаружены пропущенные значения в столбцах `days_employed` (трудовой стаж в днях) и `total_income` (ежемесячный доход).
В реальной ситуации было бы логично обратиться к источнику данных и уточнить, по возможности, причину пропусков. В учебном проекте заполним значения медианой, как более устойчивым к выбросам показателем. Предобработка столбца `total_income` выполнена, тем же методом воспользуемся для обработки трудового стажа после избавления от отрицательных и слишком больших значений.

Пока продолжим предварительную обработку данных и наведём порядок в типах данных.

### Замена типа данных

После избавления от значений NaN в столбцах `days_employed` (трудовой стаж в днях) и `total_income` (ежемесячный доход) можно изменить тип данных в этих столбцах - больше нет необходимости хранить "вещественный" NaN, а избыточная точность в числе знаков после запятой только затрудняет анализ.

Чтобы выбрать подходящий тип данных для каждого из столбцов, проверим, как распределены в них значения.

In [10]:
print(f"Минимальное значение трудового стажа: {df['days_employed'].min()} дней")
print(f"Максимальное значение трудового стажа: {df['days_employed'].max()} дней")

print(f"Минимальное значение ежемесячного дохода: {df['total_income'].min()} рублей")
print(f"Максимальное значение ежемесячного дохода: {df['total_income'].max()} рублей")

Минимальное значение трудового стажа: -18388.949900568383 дней
Максимальное значение трудового стажа: 401755.40047533 дней
Минимальное значение ежемесячного дохода: 20667.26379327158 рублей
Максимальное значение ежемесячного дохода: 2265604.028722744 рублей


Проверка максимальных и минимальных значений напомнила о том, что необходимо навести порядок с трудовым стажем - отрицательным он быть не может, равно как не может превышать разумную продолжительность жизни. Прежде чем заменить отрицательные значения в столбце проверим, сколько их.

In [11]:
print(f"В таблице {df[df['days_employed'] < 0]['days_employed'].count()} отрицательных значений трудового стажа.")

В таблице 15906 отрицательных значений трудового стажа.


Отрицательных значений стажа в днях много - при этом, им соответствуют ненулевые значения дохода.
Установить природу ошибки затруднительно. Учитывая количество строк с отрицательными значениями стажа - разумно предположить ошибку при выгрузке данных. Проверим дополнительно минимальное положительное и максимальное отрицательное значение стажа.

In [12]:
print(df[df['days_employed']>0]['days_employed'].min())
print(df[df['days_employed']<0]['days_employed'].max())

328728.72060451825
-24.14163324048118


Проверка показала, что в переменной `days_employed` положительные значения превышают продолжительность жизни человека, тогда как отрицательные не выходят за эту границу и для них достаточно заменить знак.

Можем внести необходимые коррективы: избавиться от отрицательных значений, привести значения к заявленному виду (дням), дозаполнить пропуски, обнаруженные на предыдущем этапе, и изменить тип данных в переменных на целые числа.

In [13]:
# создадим функцию, которая приведёт значение в колонке к положительному количеству дней
# тут пришлось немного модифицировать функцию, потому что к этому моменту мы ещё не избавились 
# от пропусков в переменной `days_employed`.
def normalize_data(value):
    try:
        if value < 0:
            return value / (-1)
        return value / 24
    except:
        return value

df['days_employed'] = df['days_employed'].apply(normalize_data) # применим функцию ко всем значениям колонки `days_employed`

In [14]:
# заполним пропуски в столбце `days_employed` медианой для каждой подгруппы по типу дохода
median_experience_by_income_type = df.sort_values('days_employed').groupby('income_type')['days_employed'].median()

for value in df['income_type'].unique():
    df.loc[df.loc[:,'income_type'] == value, 'days_employed'] = df.loc[df.loc[:,'income_type'] == value, 'days_employed'].fillna(median_experience_by_income_type[value])
    
# проверим, сработало ли заполнение - аналогично истории с доходом
print(f"В столбце `days_employed` {df['days_employed'].isna().sum()} нулевых значений")

В столбце `days_employed` 0 нулевых значений


In [15]:
# преобразуем значения в столбцах `days_employed` и `total_income` в целые числа методом astype, дающим возможность указать
# конкретный тип переменной на выходе
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')

# проверим, сработало ли наше решение - снова обратимся к информации о df и посмотрим на первые строки таблицы
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Обнаружили две переменные, в которых данные были представленны в виде вещественных чисел: `days_employed` и `total_income`. `days_employed` потребовал дополнительной обработки - необходимо было дозаполнить оставшиеся на прошлом этапе.
Преобразовали значения в целые числа методом `astype()`, дающим возможность указать необходимый конкретный тип значения на выходе.
Дополнительно обработали значения в переменной `days_employed` - избавились от отрицательных значений и привели к заявленному содержанию - дням вместо часов.

Можем приступить к обработке дубликатов.

In [16]:
df['education'] = df['education'].str.lower()

print(df.groupby('education_id')['education'].unique())

education_id
0                 [высшее]
1                [среднее]
2    [неоконченное высшее]
3              [начальное]
4         [ученая степень]
Name: education, dtype: object


После обработки каждому значению `education_id` соответствует ровно одно значение `education`. Можно перейти к остальным столбцам.

In [17]:
for column in ['family_status', 'income_type', 'gender']:
    print(f"Уникальные значения в столбце {column}: {df[column].unique()}\n")

Уникальные значения в столбце family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения в столбце income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Уникальные значения в столбце gender: ['F' 'M' 'XNA']



В столбцах `family_status` и `income_type` скрытых дубликатов не обнаружено. В столбце `gender` присутствует значение **XNA** - проверим, сколько таких строк и чем они отличаются от остальных.

In [18]:
display(df[df['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Строка, в которой переменная `gender` принимает значение **XNA** в наборе данных одна - её можно исключить из анализа.

In [19]:
df = df[df['gender'] != 'XNA']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Выполним также проверку переменных `dob_years` (возраст клиента) и `children`(количество детей).

In [20]:
for column in ['dob_years', 'children']:
    print(f"Минимальное значение в столбце {column}: {df[column].min()}")
    print(f"Максимальное значение в столбце {column}: {df[column].max()}\n")

Минимальное значение в столбце dob_years: 0
Максимальное значение в столбце dob_years: 75

Минимальное значение в столбце children: -1
Максимальное значение в столбце children: 20



Очевидно, что возраст заёмщика не может быть равным нулю, а количество детей -1 (*хотя я могу подогнать логику под такое значение, но это слишком уж чёрный юмор*). Вызывает вопросы и слишком большое число детей: 20. Проверим, сколько таких значений в файле с данными.

In [21]:
print(f"Нулевых значений возраста в таблице: {df.loc[df.loc[:,'dob_years'] == 0, 'dob_years'].count()}")
print()
print('Распределение количества детей:')
print(df.loc[:,'children'].value_counts())

Нулевых значений возраста в таблице: 101

Распределение количества детей:
 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Предположим, что значения -1 и 20 переменной `children` - ошибки при вводе данных, и перекодируем -1 в 1, а 20 - в 2.

In [22]:
df.loc[:, 'children'] = df.loc[:, 'children'].replace({-1:1, 20:2})
print(df.loc[:,'children'].value_counts())

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Отфильтруем нулевые значения в `dob_years`.

In [23]:
df_processed = df.loc[df.loc[:,'dob_years'] > 0, :]

Проверим, удалось ли нам избавиться от неправильных значений в обоих столбцах.

In [24]:
print(f"Нулевых значений возраста в таблице: {df_processed.loc[df_processed.loc[:,'dob_years'] == 0, 'dob_years'].count()}")
print(f"Отрицательных значений количества детей в таблице: {df_processed.loc[df_processed.loc[:,'children'] < 0, 'children'].count()}")

Нулевых значений возраста в таблице: 0
Отрицательных значений количества детей в таблице: 0


Наконец, попробуем оценить существование проблем в переменной `purpose`. Подсчитаем количество уникальных значений и оценим, сколько ответов приходится на каждое из них.

In [25]:
print(f"Всего уникальных значений в purpose: {len(df_processed['purpose'].unique())}")
print('')
print(df_processed['purpose'].value_counts())

Всего уникальных значений в purpose: 38

свадьба                                   792
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     649
операции с жильем                         647
жилье                                     641
покупка жилья                             641
покупка жилья для семьи                   640
строительство собственной недвижимости    633
операции со своей недвижимостью           630
недвижимость                              630
строительство жилой недвижимости          623
покупка недвижимости                      620
строительство недвижимости                620
покупка своего жилья                      619
ремонт жилью                              610
покупка жилой недвижимости                604
на покупку своего автомобиля           

Очевидна необходимость дополнительной обработки значений в столбце `purpose` - для более подробного анализа необходимо провести лемматизацию.

**Вывод**

В процессе обработки дубликатов обнаружили и исправили расхождение регистров записи в переменной `education`, нашли уникальное значение **XNA** (вероятный пропуск) в переменной `gender` - строку с этим значением отфильтровали. Избавились от некоторого количества строк с нулевым возрастом и отрицательным количеством детей.
Подтвердили необходимость проведения лемматизации значений в столбце `purpose` для продолжения анализа и ответов на поставленные вопросы.

### Лемматизация

Проведение лемматизации необходимо для того, чтобы сгруппировать значения в переменной `purpose`. На первый взгляд видны несколько общих позиций:
* Получение образования
* Покупка автомобиля
* Сделки с недвижимостью (коммерческой и жилой - эти две позиции стоит разделять)
* Свадьба

Ручная обработка для перегруппировки 38 уникальных значений возможна, но при существовании автоматических инструментов неоправданна.
Сохраним лемматизированные строки из переменной `purpose` для каждой строки в новую переменную `purpose_list`.

In [26]:
df_processed.loc[:, 'purpose_list'] = df['purpose'].apply(m.lemmatize)
print(df_processed['purpose_list'])

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose_list, Length: 21423, dtype: object


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Необходимо также оценить, сколько значений каждой леммы получилось в наборе данных.
Для этого стоит составить список всех лемм, встречающихся в получившихся списках. 

In [27]:
# небольшой чит - методу explode() нас ещё не учили, но он удобен - разворачивает список.
print(Counter(df_processed['purpose_list'].explode()))

Counter({' ': 33541, '\n': 21423, 'недвижимость': 6343, 'покупка': 5884, 'жилье': 4449, 'автомобиль': 4293, 'образование': 4004, 'с': 2910, 'операция': 2599, 'свадьба': 2334, 'свой': 2228, 'на': 2221, 'строительство': 1876, 'высокий': 1367, 'получение': 1311, 'коммерческий': 1310, 'для': 1291, 'жилой': 1227, 'сделка': 941, 'дополнительный': 905, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 651, 'семья': 640, 'собственный': 633, 'со': 630, 'ремонт': 610, 'подержанный': 487, 'подержать': 479, 'приобретение': 460, 'профильный': 435})


Полученный список значений позволяет провести категоризацию данных.

**Вывод**

Провели лемматизацию значений переменной `purpose`, на выходе для каждой строки получили список с леммами, хранящийся в переменной `purpose_list`. После категоризации данных исходны и новый столбцы можно будет удалить.

### Категоризация данных

Сохранённые в `purpose_list` списки дают нам возможность категоризировать цели кредита. Для этого напишем функцию, которая на вход будет получать список `purpose_list` для каждой строки и на его основе присваивать ту или иную категорию.

Определимся с набором категорий:
* Образование
* Автомобиль
* Свадьба
* Жилая недвижимость (включая ремонт)
* Коммерческая и другая недвижимость

Номера категорий определим на отрезке [1:5].

Предусмотрим также значение для непредусмотренных случаев - **Other**, такому значению будет соответствовать NaN для облегчения оценки качества категоризации.

In [28]:
def categorize(row):
    if 'образование' in row['purpose_list']:
        return 'образование'
    if 'автомобиль' in row['purpose_list']:
        return 'автомобиль'
    if 'свадьба' in row['purpose_list']:
        return 'свадьба'
    if 'жилье' in row['purpose_list'] or 'недвижимость' in row['purpose_list']:
        if 'жилье' in row['purpose_list'] or 'жилой' in row['purpose_list']:
            return 'жилая недвижимость'
        return 'коммерческая и другая недвижимость'
    return float('nan')
    
df_processed.loc[:, 'purpose_category'] = df_processed.apply(categorize, axis = 1)
print(df_processed['purpose_category'].value_counts())

жилая недвижимость                    5676
коммерческая и другая недвижимость    5116
автомобиль                            4293
образование                           4004
свадьба                               2334
Name: purpose_category, dtype: int64


Присвоим получившимся категориям заранее определённые коды (в порядке их определения).

In [29]:
def encode(row):
    if 'образование' in row['purpose_category']:
        return 1
    if 'автомобиль' in row['purpose_category']:
        return 2
    if 'свадьба' in row['purpose_category']:
        return 3
    if 'жилая недвижимость' in row['purpose_category']:
        return 4
    if 'коммерческая и другая недвижимость' in row['purpose_category']:
        return 5
    return float('nan')

df_processed.loc[:, 'purpose_id'] = df_processed.apply(encode, axis = 1)
df_processed.drop(labels = ['purpose', 'purpose_list'], axis = 1)
print(df_processed['purpose_id'].value_counts())

4    5676
5    5116
2    4293
1    4004
3    2334
Name: purpose_id, dtype: int64


Исходя из вопросов, поставленных в исследовании, категоризируем также переменную `children` - в исходном виде в переменной сохранено количество детей, данные можно сгруппировать: 1 ребёнок, 2 или больше детей.

In [30]:
def children_presence(row):
    if row['children'] == 0:
        return 'no children'
    elif row['children'] <= 2:
        return '1-2 children'
    else:
        return '3 or more children'

df_processed.loc[:, 'children_exist'] = df_processed.apply(children_presence, axis = 1)
print(df_processed['children_exist'].value_counts())

no children           14079
1-2 children           6966
3 or more children      378
Name: children_exist, dtype: int64


Также нам необходимо категоризировать значения уровня дохода, чтобы ответить на вопрос о влиянии дохода на возврат кредита в срок. Для начала оценим разброс значений в столбце - проверим минимальное отличное от нуля значение, медиану, среднее и максимум.

In [31]:
print('Минимальное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].min())
print('Медианное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].sort_values().median())
print('Среднее значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].mean())
print('Максимальное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].max())

Минимальное значение ненулевого дохода:  20667
Медианное значение ненулевого дохода:  142594.0
Среднее значение ненулевого дохода:  165263.42375017505
Максимальное значение ненулевого дохода:  2265604


Среднее значение ненулевого дохода несколько завышено - для среднего значения дохода это довольно стандартная ситуация.

Воспользуемся медианой, а также дополнительно посчитаем первый и третий квартиль, чтобы разделить выборку на 4 группы по доходу:
* `total_income` < первого квартиля - низкий доход
* Первый квартиль <= `total_income` < медианы - ниже среднего
* Медианное значение <= `total_income` < третьего квартиля - выше среднего
* `total income` > третьего квартиля - высокий доход

In [32]:
income_median = df_processed.loc[df_processed.loc[:, 'total_income'] > 0 , 'total_income'].sort_values().median()
income_first_quartile = df_processed.loc[df_processed.loc[:, 'total_income'] > 0 , 'total_income'].sort_values().quantile(0.25)
income_third_quartile = df_processed.loc[df_processed.loc[:, 'total_income'] > 0 , 'total_income'].sort_values().quantile(0.75)

def income_cat(row):
    if row['total_income'] < income_first_quartile:
        return 'Низкий доход'
    elif row['total_income'] < income_median:
        return 'Ниже среднего'
    elif row['total_income'] < income_third_quartile:
        return 'Выше среднего'
    return 'Высокий доход'

df_processed.loc[:,'income_category'] = df_processed.apply(income_cat, axis = 1)
display(df_processed.loc[:,('total_income', 'income_category')])

,total_income,income_category
0,253875,Высокий доход
1,112080,Ниже среднего
2,145885,Выше среднего
3,267628,Высокий доход
4,158616,Выше среднего
...,...,...
21520,224791,Высокий доход
21521,155999,Выше среднего
21522,89672,Низкий доход
21523,244093,Высокий доход


**Вывод**

Лемматизировали переменную `purpose`, выделили 6 категорий кредитов, сохранив их в `purpose_category`. Каждой категории присвоили номер и сохранили его в `purpose_id`.

Категоризировали наличие детей и показатель дохода.

Можно приступать к следующему шагу: ответить на вопросы исследования.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
children_pivot = df_processed.pivot_table(index = 'children_exist', values = 'debt', aggfunc = ['count', 'sum'])
children_pivot['ratio'] = children_pivot['sum'] / children_pivot['count'] * 100
display(children_pivot)

,count,sum,ratio
,debt,debt,
children_exist,,,
1-2 children,6966,644,9.244904
3 or more children,378,31,8.201058
no children,14079,1058,7.514738


**Вывод**

Среди клиентов с одним-двумя детьми около 9% имеют просроченный кредит - вне зависимости от количества детей, тогда как среди заёмщиков без детей доля просрочек составляет 7.5%. Любопытно, что среди заёмщиков с 3 и более детьми процент просроченных кредитов ниже, чем среди тех, у кого 1-2 ребёнка - хотя размер группы небольшой. Зависимость между наличием детей и возвратом кредита в срок существует *(хорошо бы проверить результат z-тестом, например, но с учётом размера выборки 1.5 п.п. разницы на таком удалении от 50% вероятно будут значимы)*.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
family_status_pivot = df_processed.pivot_table(index = 'family_status', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'family_status', values = 'debt', aggfunc = 'count') * 100
display(family_status_pivot)

,debt
family_status,
Не женат / не замужем,9.760458
в разводе,7.172996
вдовец / вдова,6.492147
гражданский брак,9.290012
женат / замужем,7.517638


**Вывод**

Наиболее высокий процент невозврата кредита в срок наблюдается у клиентов, не состоящих в браке или сожительствующих (*термин "гражданский брак" неудачный - он описывает собственно зарегистрированный в органах ЗАГС брак, в противоположность или дополнение церковному*).

Среди разведённых и находящихся в браке доля должников по кредитам около 7%, а рекордсмены по возвратам - вдовцы и вдовы, здесь доля просроченных платежей составляет 6.5%.

Зависимость между семейным положением и возвратом кредита в срок есть (*учтём здесь, что на возврат кредита в срок может влиять возраст заёмщика - с высокой долей вероятности, именно молодёжь не состоит в браке или сожительствует, тогда как в остальных группах средний возраст весьма вероятно выше*).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
income_pivot = df_processed.pivot_table(index = 'income_category', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'income_category', values = 'debt', aggfunc = 'count') * 100
display(income_pivot)

,debt
income_category,
Высокий доход,7.188200
Выше среднего,8.551069
Ниже среднего,8.712466
Низкий доход,7.935026


**Вывод**

Согласно предоставленным данным, зависимость между доходом и возвратом кредита в срок есть, но она нелинейная. Заёмщики с высоким доходом имеют просроченный кредит в ~7.2% случаев, заёмщики со средним доходом - в ~7.9%.

Наибольшая доля просроченного кредита наблюдается в группе с доходом ниже среднего: 8.7% заёмщиков из этой группы имеют задолженность по кредиту. Чуть лучше, хоть и незначительно, ситуация среди тех, чей доход выше среднего - здесь доля задолженностей составляет 8.55%

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
purpose_pivot = df_processed.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = 'count') * 100
display(purpose_pivot)

,debt
purpose_category,
автомобиль,9.317494
жилая недвижимость,6.959126
коммерческая и другая недвижимость,7.505864
образование,9.240759
свадьба,7.883462


**Вывод**

Кредиты на жилую недвижимость реже других бывают просрочены - лишь в около 7% случаев, также в лидерах по "дисциплине" заёмщика - кредиты на коммерческую недвижимость (7.5%).

Доля просроченных кредитов на свадьбу составляет 7.9% - мы уже знаем, что люди, состоящие в браке, более склонны платить вовремя.

Антилидеры рейтинга - кредиты на автомобиль и образование. В обоих случаях доля просроченных кредитов превышает 9%. Здесь я радуюсь за себя - на обучение в Практикуме брать кредит не пришлось, а до получения прав ещё несколько месяцев. По крайней мере, в ближайшее время мне не грозит оказаться тем единственным должником из 11 заёмщиков.

## Шаг 4. Общий вывод

В ходе исследования надёжности заёмщиков мы провели предварительную обработку предоставленных данных и проверили наличие зависимости между некоторыми характеристиками заёмщиков (либо самих кредитов) и возвратом кредита в срок.

По каждому показателю зависимость так или иначе обнаружена:
* Невозврат кредита в срок более распространён среди тех заёмщиков, кто ещё не создал полноценную семью и либо сожительствует с партнёром, либо не состоит в постоянных отношениях.
* При этом, дети в семье делают возврат кредита в срок более сложной задачей - доля просроченных кредитов среди заёмщиков с 1-2 детьми выше, чем среди бездетных, на ~1.7 процентных пункта. Семьи с 3 или более детьми более ответственны в выплате кредитов: в этой подгруппе просроченны 8.2% кредитов (vs. 9.2% в семьях с 1-2 детьми, 7.5% у заёмщиков без детей). 
* Заёмщики с высоким доходом более склонны возвращать кредиты в срок в сравнение с заёмщиками со средними или низкими доходами. Кроме того, люди с низким доходом более ответственны, чем люди с доходом выше или ниже среднего. Возможно, условный средний класс может позволить себе брать более дорогие кредиты на большие суммы, чем люди с низким доходом, что при определённых условиях может приводить к неспособности выплачивать кредит вовремя.
* Самые безопасные для банка кредиты связаны с недвижимостью: кредиты взятые покупку жилья или коммерческой недвижимости (вероятно, в силу обеспечения этой самой недвижимостью) возвращаются лучше, чем кредиты на автомобиль или образование.